%md
## Categories

In [0]:
bronze_categories_df = (
    spark.read
         .format("delta")
         .load("/Volumes/sales_databricks_workspace/bronze/bronze_retail_vol/sales/3a_superstore/categories/")
)


In [0]:
categories_df = (
    bronze_categories_df
    .filter(col("ITEMID").isNotNull())
    .dropDuplicates(["ITEMID"])
)


In [0]:
categories_df = (
    categories_df
    .withColumn("CATEGORY1", upper(trim(col("CATEGORY1"))))
    .withColumn("CATEGORY2", upper(trim(col("CATEGORY2"))))
    .withColumn("CATEGORY3", upper(trim(col("CATEGORY3"))))
    .withColumn("CATEGORY4", upper(trim(col("CATEGORY4"))))
    .withColumn("CATEGORY2_ID", upper(trim(col("CATEGORY2_ID"))))
    .withColumn("CATEGORY3_ID", upper(trim(col("CATEGORY3_ID"))))
    .withColumn("CATEGORY4_ID", upper(trim(col("CATEGORY4_ID"))))
    .withColumn("BRAND", upper(trim(col("BRAND"))))
    .withColumn("ITEMNAME", upper(trim(col("ITEMNAME"))))
)


In [0]:
categories_df = (
    categories_df
    .withColumnRenamed("ingestion_ts", "INGESTION_TS")
    .withColumnRenamed("ITEMID", "ITEM_ID")
    .withColumnRenamed("ITEMCODE", "ITEM_CODE")
    .withColumnRenamed("ITEMNAME", "ITEM_NAME")

    .drop("source_file")
)


In [0]:
categories_df.display()

In [0]:
categories_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("sales_databricks_workspace.silver.categories")


In [0]:
%sql
select * from silver.categories

## Orders



In [0]:
bronze_orders_df = (
    spark.read
         .format("delta")
         .load("/Volumes/sales_databricks_workspace/bronze/bronze_retail_vol/sales/3a_superstore/orders/")
)


In [0]:
orders_df = (
    bronze_orders_df
    .filter(col("ORDERID").isNotNull())
    .filter(col("USERID").isNotNull())
    .filter(col("BRANCH_ID").isNotNull())
    .filter(col("DATE_").isNotNull())
)


In [0]:
orders_df = orders_df.dropDuplicates(["ORDERID"])


In [0]:
orders_df = orders_df.withColumn(
    "TOTALBASKET",
    regexp_replace(col("TOTALBASKET"), ",", ".")
)


In [0]:
orders_df = orders_df.withColumn(
    "TOTALBASKET",
    col("TOTALBASKET").cast("double")
)


In [0]:
orders_df = orders_df.withColumn(
    "TOTALBASKET",
    round(col("TOTALBASKET").cast(DecimalType(12, 2)), 2)
)


In [0]:
orders_df = orders_df.filter(col("TOTALBASKET").isNotNull())


In [0]:
orders_df = (
    orders_df
    .withColumnRenamed("ingestion_ts", "INGESTION_TS")
    .drop("source_file")
)


In [0]:
orders_df = orders_df.withColumnRenamed("NAMESURNAME", "FULL_NAME")
orders_df = orders_df.withColumnRenamed("ORDERID", "ORDER_ID")
orders_df = orders_df.withColumnRenamed("DATE_", "DATE")
orders_df = orders_df.withColumnRenamed("USERID", "USER_ID")
orders_df = orders_df.withColumnRenamed("TOTALBASKET", "TOTAL_BASKET")


In [0]:
orders_df.display()

In [0]:
orders_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("sales_databricks_workspace.silver.orders")


In [0]:
%sql
select * from silver.orders

%md
## Order_details


In [0]:
bronze_order_details_df = (
    spark.read
         .format("delta")
         .load("/Volumes/sales_databricks_workspace/bronze/bronze_retail_vol/sales/3a_superstore/order_details/")
)


In [0]:
order_details_df = (
    bronze_order_details_df
    .filter(col("ORDERDETAILID").isNotNull())
    .filter(col("ORDERID").isNotNull())
    .filter(col("ITEMID").isNotNull())
    .filter(col("AMOUNT") > 0)
)


In [0]:
order_details_df = order_details_df.dropDuplicates(["ORDERDETAILID"])


In [0]:
order_details_df = order_details_df.withColumn(
    "UNITPRICE",
    regexp_replace(col("UNITPRICE"), ",", ".")
)


In [0]:
order_details_df = order_details_df.withColumn(
    "UNITPRICE",
    col("UNITPRICE").cast(DecimalType(12, 2))
)


In [0]:
order_details_df = order_details_df.withColumn(
    "TOTALPRICE",
    regexp_replace(col("TOTALPRICE"), ",", ".")
)


In [0]:
order_details_df = order_details_df.withColumn(
    "TOTALPRICE",
    col("TOTALPRICE").cast(DecimalType(12, 2))
)


In [0]:
order_details_df = order_details_df.filter(
    col("UNITPRICE").isNotNull() & col("TOTALPRICE").isNotNull()
)


In [0]:
order_details_df = (
    order_details_df
    .withColumnRenamed("ORDERID", "ORDER_ID")
    .withColumnRenamed("ORDERDETAILID", "ORDER_DETAIL_ID")
    .withColumnRenamed("UNITPRICE", "UNIT_PRICE")
    .withColumnRenamed("TOTALPRICE", "TOTAL_PRICE")
    .withColumnRenamed("ITEMID", "ITEM_ID")
    .withColumnRenamed("ITEMCODE", "ITEM_CODE")
    .withColumnRenamed("ingestion_ts", "INGESTION_TS")
)


In [0]:
order_details_df = order_details_df.drop("source_file")


In [0]:
order_details_df.display()

In [0]:
order_details_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("sales_databricks_workspace.silver.order_details")


In [0]:
%sql
select * from silver.order_details